# SSP emissions runs using CMIP6 calibrations

This notebook gives an example of running SSP scenarios in FaIR using climate response calibrations from 66 CMIP6 models for a total of 8$\times$66 = 528 ensemble members in parallel.

We will run stochastic mode to attempt to capture internal variability.

In [ ]:
import copy

import numpy as np
import pandas as pd

from fair21 import FAIR
from fair21.io import read_properties
from fair21.interface import fill, initialise
from fair21.earth_params import seconds_per_year

In [ ]:
f = FAIR()

In [ ]:
# create world running from 1850 to 2100, at 1-year intervals
f.define_time(1850, 2100, 1)

In [ ]:
# Define SSP scenarios
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']
f.define_scenarios(scenarios)

In [ ]:
df = pd.read_csv("../data/calibration/4xCO2_cummins.csv")
models = df['model'].unique()
configs = []

for imodel, model in enumerate(models):
    for run in df.loc[df['model']==model, 'run']:
        configs.append(f"{model}_{run}")
f.define_configs(configs)

In [ ]:
properties = read_properties()
species = list(properties.keys())

In [ ]:
f.define_species(species, properties)

In [ ]:
f.run_control(ch4_method='Thornhill2021', aci_method='Stevens2015')

In [ ]:
f.allocate()

In [ ]:
f.run_control(ch4_method='Thornhill2021')

In [ ]:
f.aci_method  # sort out overwriting

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
df_forc = pd.read_csv('../data/rcmip/rcmip-radiative-forcing-annual-means-v5-1-0.csv')

In [ ]:
species_to_rcmip = {specie: specie.replace("-", "") for specie in species}
species_to_rcmip['CO2 FFI'] = 'CO2|MAGICC Fossil and Industrial'
species_to_rcmip['CO2 AFOLU'] = 'CO2|MAGICC AFOLU'
species_to_rcmip['NOx aviation'] = 'NOx|MAGICC Fossil and Industrial|Aircraft'
for specie in ['CO2', 'Solar', 'Volcanic', 'Aerosol-radiation interactions', 'Aerosol-cloud interactions', 'Ozone',
               'Contrails', 'Light absorbing particles on snow and ice', 'Stratospheric water vapour', 'Land use']:
    del(species_to_rcmip[specie])
#species_to_rcmip

In [ ]:
desired_emissions_units = {specie: f'kt {specie.replace("-", "")}/yr' for specie in species}
desired_emissions_units['CO2 FFI'] = 'Gt CO2/yr'
desired_emissions_units['CO2 AFOLU'] = 'Gt CO2/yr'
desired_emissions_units['CH4'] = 'Gt CO2/yr'
desired_emissions_units['N2O'] = 'Mt N2O/yr'
desired_emissions_units['Sulfur'] = 'Mt SO2/yr'
desired_emissions_units['BC'] = 'Mt BC/yr'
desired_emissions_units['OC'] = 'Mt OC/yr'
desired_emissions_units['NH3'] = 'Mt NH3/yr'
desired_emissions_units['NOx'] = 'Mt NO2/yr'
desired_emissions_units['NOx aviation'] = 'Mt NO2/yr'
desired_emissions_units['VOC'] = 'Mt VOC/yr'
desired_emissions_units['CO'] = 'Mt CO/yr'

for specie in ['CO2', 'Solar', 'Volcanic', 'Aerosol-radiation interactions', 'Aerosol-cloud interactions', 'Ozone',
               'Contrails', 'Light absorbing particles on snow and ice', 'Stratospheric water vapour', 'Land use']:
    del(desired_emissions_units[specie])
desired_emissions_units

In [ ]:
compound_convert = {}
df = pd.read_csv('../src/fair21/defaults/data/ar6/species_configs_properties.csv', index_col=0)
species_molwts = {specie.replace("-",""): value for specie, value in dict(df['molecular_weight']).items()}

species_molwts['C'] = 12.011
species_molwts['NO2'] = 46.006
species_molwts['NO'] = 30.006
species_molwts['N'] = 14.007
species_molwts['N2'] = 28.014
species_molwts['SO2'] = 64.069
species_molwts['S'] = 32.07

for specie_from, mw_from in species_molwts.items():
     if ~np.isnan(mw_from):
        compound_convert[specie_from.replace("-", "")] = {}
        for specie_to, mw_to in species_molwts.items():
            if ~np.isnan(mw_to):
                compound_convert[specie_from.replace("-", "")][specie_to] = mw_to/mw_from
compound_convert

In [ ]:
time_convert = {
    'yr': {
        'yr': 1,
        'day': 1 / (seconds_per_year / 3600 / 24),
        's': 1 / seconds_per_year,
    },
    'day' : {
        'yr': seconds_per_year / 3600 / 24,
        'day': 1,
        's': 3600 * 24,
    },
    's' : {
        'yr': seconds_per_year,
        'day': 1 / 3600 / 24,
        's': 1,
    }
}

In [ ]:
prefix_convert = {
    'Mt': {
        'Gt': 0.001,
        'Tg': 1,
        'Mt': 1,
        'kt': 1000,
        't': 1e6,
        'kg': 1e9,
        'g': 1e12
    },
    'kt': {
        'Gt': 1e-6,
        'Tg': 1e-3,
        'Mt': 1e-3,
        'kt': 1,
        't': 1e3,
        'kg': 1e6,
        'g': 1e9
    },
}

In [ ]:
specie = 'CO2 FFI'
unit = df_emis.loc[
    (df_emis['Scenario']=='ssp119') & (df_emis['Variable'].str.endswith(species_to_rcmip[specie])) & 
    (df_emis['Region']=='World'),
'Unit'].values[0]#.interpolate(axis=1).values.squeeze()
prefix_convert[unit.split()[0]][desired_emissions_units[specie].split()[0]]

In [ ]:
compound_convert[unit.split()[1].split('/')[0]][desired_emissions_units[specie].split()[1].split('/')[0]]

In [ ]:
time_convert[unit.split()[1].split('/')[1]][desired_emissions_units[specie].split()[1].split('/')[1]]

In [ ]:
# for specie, specie_rcmip_name in species_to_rcmip.items():
#     unit = df_emis.loc[
#         (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+specie_rcmip_name)) & 
#         (df_emis['Region']=='World'), 'Unit'
#         ].values[0]
#     print(unit)
#     print(prefix_convert[unit.split()[0]][desired_emissions_units[specie].split()[0]])

In [ ]:
df['baseline_concentration']['CO2']

#properties['CO2 FFI']

In [ ]:
for scenario in scenarios:
    for specie, specie_rcmip_name in species_to_rcmip.items():
        emis_in = df_emis.loc[
            (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+specie_rcmip_name)) & 
            (df_emis['Region']=='World'), str(f.timebounds[0]):str(f.timebounds[-2])
        ].interpolate(axis=1).values.squeeze()
        # TODO: raise error if can't find
        
        unit = df_emis.loc[
            (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+specie_rcmip_name)) & 
            (df_emis['Region']=='World'), 'Unit'
        ].values[0]

        emis_in = emis_in * (
            prefix_convert[unit.split()[0]][desired_emissions_units[specie].split()[0]] *
            compound_convert[unit.split()[1].split('/')[0]][desired_emissions_units[specie].split()[1].split('/')[0]] *
            time_convert[unit.split()[1].split('/')[1]][desired_emissions_units[specie].split()[1].split('/')[1]]
        )

        fill(f.emissions, emis_in[:, None], specie=specie, scenario=scenario)
        
    for specie in ['Volcanic', 'Solar']:
        forc_in = df_forc.loc[
            (df_forc['Scenario']==scenario) & (df_forc['Variable'].str.endswith("|"+specie)) & 
            (df_forc['Region']=='World'), str(f.timebounds[0]):str(f.timebounds[-1])
        ].interpolate(axis=1).values.squeeze()
        fill(f.forcing, forc_in[:, None], specie=specie, scenario=scenario)

for specie in species:
    if df.loc[specie, 'greenhouse_gas']:
        initialise(f.concentration, df['baseline_concentration'][specie], specie=specie)
        
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
#f.concentration[0,0,0,:]

In [ ]:
f.fill_species_configs()

In [ ]:
f.climate_configs

In [ ]:
df = pd.read_csv("../data/calibration/4xCO2_cummins.csv")
models = df['model'].unique()

seed = 1355763

for config in configs:
    model, run = config.split('_')
    condition = (df['model']==model) & (df['run']==run)
    fill(f.climate_configs['ocean_heat_capacity'], df.loc[condition, 'C1':'C3'].values.squeeze(), config=config)
    fill(f.climate_configs['ocean_heat_transfer'], df.loc[condition, 'kappa1':'kappa3'].values.squeeze(), config=config)
    fill(f.climate_configs['deep_ocean_efficacy'], df.loc[condition, 'epsilon'].values[0], config=config)
    fill(f.climate_configs['gamma_autocorrelation'], df.loc[condition, 'gamma'].values[0], config=config)
    fill(f.climate_configs['sigma_eta'], df.loc[condition, 'sigma_eta'].values[0], config=config)
    fill(f.climate_configs['sigma_xi'], df.loc[condition, 'sigma_xi'].values[0], config=config)
    fill(f.climate_configs['stochastic_run'], True, config=config)
    fill(f.climate_configs['use_seed'], True, config=config)
    fill(f.climate_configs['seed'], seed, config=config)
    
    seed = seed + 399

In [ ]:
f.climate_configs

## Run FaIR

In [ ]:
f.climate_configs['use_seed'][0]

In [ ]:
f.run()

In [ ]:
f.ebms

In [ ]:
import matplotlib.pyplot as pl

## Make some nice plots

In [ ]:
pl.plot(f.timebounds, f.temperature.loc[dict(scenario='ssp119', layer=0)], label=f.configs);
#pl.title('Ramp scenario: temperature')
#pl.xlabel('year')
#pl.ylabel('Temperature anomaly (K)')
#pl.legend()

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16, 10))

for i in range(8):
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.min(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        np.max(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 5, axis=1), 
        np.percentile(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 16, axis=1), 
        np.percentile(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].plot(
        np.arange(1850.5, 2101), 
        np.median(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        color='#000000',
    )
    ax[i//4,i%4].set_xlim(1850,2100)
    ax[i//4,i%4].set_ylim(-1, 10)
    ax[i//4,i%4].axhline(0, color='k', ls=":", lw=0.5)
    ax[i//4,i%4].set_title(scenarios_to_include[i])
pl.suptitle('Temperature anomaly')

In [ ]:
fair.calculate_ocean_heat_content_change()

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16, 10))

for i in range(8):
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.min(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        np.max(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), 5, axis=1), 
        np.percentile(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), 16, axis=1), 
        np.percentile(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].plot(
        np.arange(1850.5, 2101), 
        np.median(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        color='#000000',
    )
    ax[i//4,i%4].set_xlim(1850,2100)
    #ax[i//4,i%4].set_ylim(-1, 10)
    ax[i//4,i%4].axhline(0, color='k', ls=":", lw=0.5)
    ax[i//4,i%4].set_title(scenarios_to_include[i])

pl.suptitle('Ocean heat content change')

In [ ]:
# Squirrel this away as a TODO to check ozone forcing
fair.forcing_array[269, 2, :, 54, 0]

In [ ]:
fair.species_index_mapping

In [ ]:
scenarios[0].name

In [ ]:
fair.scenarios[0].list_of_species[51].species_id.name

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16, 10))

for i in range(8):
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.min(fair.scenarios[i].list_of_species[51].concentration[100:, :], axis=1), 
        np.max(fair.scenarios[i].list_of_species[51].concentration[100:, :], axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.scenarios[i].list_of_species[51].concentration[100:, :], 5, axis=1), 
        np.percentile(fair.scenarios[i].list_of_species[51].concentration[100:, :], 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.scenarios[i].list_of_species[51].concentration[100:, :], 16, axis=1), 
        np.percentile(fair.scenarios[i].list_of_species[51].concentration[100:, :], 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].plot(
        np.arange(1850.5, 2101), 
        np.median(fair.scenarios[i].list_of_species[51].concentration[100:, :], axis=1), 
        color='#000000',
    )
    ax[i//4,i%4].set_xlim(1850,2100)
    ax[i//4,i%4].set_title(scenarios_to_include[i])
pl.suptitle('CO2 concentrations')

In [ ]:
pl.plot(fair.toa_imbalance[:,7,0,0,0])

In [ ]:
pl.plot(fair.forcing_sum_array[:,7,0,0,0])
fair.forcing_sum_array[-1,7,0,0,0]

In [ ]:
fair.forcing_sum_array[270,7,0,0,0]

In [ ]:
pl.plot(fair.temperature[:, 7, 0, 0, 0])
pl.plot(fair.temperature[:, 7, 0, 0, 1])
pl.plot(fair.temperature[:, 7, 0, 0, 2])